# FTX reader

In [1]:
import matplotlib
%matplotlib inline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px
import nest_asyncio
nest_asyncio.apply()
import os,sys
sys.path.append('../')
from io_utils import *
from ftx_history import *

end_time=datetime.now(timezone.utc)#.replace(minute=0,second=0,microsecond=0)
start_time=datetime(2021,12,2)#end_time-timedelta(hours=1)
halflife=timedelta(days=1)
modes=[('h','high'),('c','close'),('l','low'),('v','size')]

In [2]:
coins = pd.read_excel('.\Runtime\ApprovedRuns\current_weights.xlsx').sort_values(by='optimalWeight',key=lambda f: np.abs(f),ascending=False)
coins = list(coins.set_index(['name']).drop(['USD','total']).head(5).index.map(lambda f: f.replace('-PERP','')))

In [3]:
futures_df=ftx_history_main('get',coins,'ftx',7)

### rates markets

In [4]:
selected = [df for df in futures_df.columns if any(df.split('-')[0].split('/')[0]==coin for coin in (coins+['USD']))]

prices=futures_df[[c for c in selected if any(string in c for string in ['price/c'])]]

short_rates=futures_df[[c for c in selected if any(string in c for string in ['PERP/rate/c','rate/borrow'])]]
short_rates=short_rates.ewm(halflife=halflife,times=short_rates.index).mean()

borrow_size=futures_df[[c for c in selected if any(string in c for string in ['/rate/size','price/volume'])]]

for c in coins:
    long_perp_carry=short_rates[c+'-PERP/rate/c']-short_rates['USD/rate/borrow']
    short_perp_carry=-short_rates[c+'-PERP/rate/c']-short_rates[c+'/rate/borrow']
    short_rates[c+'/best_perp_carry']= pd.concat([long_perp_carry, short_perp_carry]).max(level=0)
   # long_rates[c+'/long_future_carry']=long_rates[c+'-1231/rate/c']-short_rates['USD/rate/borrow']
   # long_rates[c+'/short_future_carry']=-long_rates[c+'-1231/rate/c']-short_rates[c+'/rate/borrow']

pd.concat([short_rates.filter(like='best_perp_carry')],axis=0).iplot()#))

C:\Users\david\AppData\Local\Temp/ipykernel_48136/577199697.py:13: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().

C:\Users\david\AppData\Local\Temp/ipykernel_48136/577199697.py:13: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().

C:\Users\david\AppData\Local\Temp/ipykernel_48136/577199697.py:13: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().

C:\Users\david\AppData\Local\Temp/ipykernel_48136/577199697.py:13: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated

In [5]:
pd.concat([short_rates.filter(like='borrow')],axis=0).iplot()#))

In [6]:
pd.concat([borrow_size],axis=0).iplot()#))

### carry directionality

In [ ]:
plotthat = futures_df[['BNB-PERP/rate/h',
       'BNB-PERP/rate/l', 'BNB-PERP/rate/c']]
plotthat.columns=['h','l','c']

In [ ]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=plotthat.index,#+x_rev,
    y=0.5*(plotthat['l'].values+plotthat['h'].values),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='BNB',
))
fig.add_trace(go.Scatter(
    x=plotthat.index, y=plotthat['c'].values,
    line_color='rgb(0,100,80)',
    name='Fair',
))

fig.update_traces(mode='lines')
fig.show()

In [ ]:
pd.concat([prices,short_rates.filter(like='carry')],axis=0).iplot(secondary_y=list(prices.columns))

# stopout: show min/max of premiums

In [ ]:
end_time=datetime.now(timezone.utc)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(hours=1)
granular_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='15s',coin_list=coins)

In [ ]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compute_premium_vs_spot(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/price/'+modes[mode][0]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [ ]:
granular_df['premium']=compute_premium_vs_spot(granular_df)
prices=granular_df[[c+'/price/c' for c in coins]]
granular_df[['premium']].iplot()

In [ ]:
premium=compute_premium(granular_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
pd.concat([prices,ewMax],axis=0).iplot(secondary_y=list(prices.columns))

In [ ]:
premium=compute_premium(granular_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
pd.concat([prices,ewMin],axis=0).iplot(secondary_y=list(prices.columns))